# Improved Hidden Markov Model for Hangman

## Improvements over Basic HMM:
1. **N-gram Transition Probabilities** - Model letter sequences
2. **Enhanced Pattern Matching** - Better handling of partial words
3. **Adaptive Smoothing** - Handle unseen patterns gracefully
4. **Context-Aware Predictions** - Use bigram/trigram context
5. **Vowel/Consonant Balance** - Strategic letter selection

**Goal:** Improve from 19.9% to 30%+ while keeping HMM framework

In [ ]:
import numpy as np
import pandas as pd
import pickle
from collections import Counter, defaultdict
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
print("Libraries imported successfully!")

## 1. Load Data

In [ ]:
# Load and clean data
def load_and_clean_data(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        words = []
        for line in f:
            word = line.strip().lower()
            word = ''.join(c for c in word if c.isalpha())
            if len(word) >= 2:
                words.append(word)
    return words

corpus = load_and_clean_data('Data/Data/corpus.txt')
test_words = load_and_clean_data('Data/Data/test.txt')

print(f"Corpus: {len(corpus)} words ({len(set(corpus))} unique)")
print(f"Test: {len(test_words)} words ({len(set(test_words))} unique)")
print(f"Overlap: {len(set(corpus) & set(test_words))} words")

## 2. Improved HMM with N-gram Features

### Key Improvements:
- **Emission Probabilities:** Position-based letter frequencies (original HMM)
- **Transition Probabilities:** Bigram/trigram patterns (NEW)
- **Smoothing:** Laplace smoothing for unseen patterns (NEW)
- **Context:** Use revealed letters for predictions (ENHANCED)

In [ ]:
class ImprovedHMM:
    """Enhanced HMM with N-gram transition probabilities"""
    
    def __init__(self, smoothing=0.1):
        # Emission probabilities (position-based)
        self.emission_prob = {}  # P(letter | position, word_length)
        
        # Transition probabilities (sequence-based)
        self.bigram_prob = defaultdict(Counter)  # P(letter_i+1 | letter_i)
        self.trigram_prob = defaultdict(lambda: defaultdict(Counter))  # P(letter_i+2 | letter_i, letter_i+1)
        
        # Overall frequencies
        self.unigram_freq = Counter()
        self.length_freq = {}
        
        self.smoothing = smoothing
        self.vowels = set('aeiou')
        
    def train(self, words):
        """Train HMM on corpus"""
        print("Training Improved HMM...")
        
        for word in tqdm(words, desc="Training"):
            word_len = len(word)
            
            # Initialize emission probabilities
            if word_len not in self.emission_prob:
                self.emission_prob[word_len] = {}
                self.length_freq[word_len] = Counter()
            
            # Emission probabilities: P(letter | position)
            for pos, letter in enumerate(word):
                if pos not in self.emission_prob[word_len]:
                    self.emission_prob[word_len][pos] = Counter()
                self.emission_prob[word_len][pos][letter] += 1
                self.unigram_freq[letter] += 1
                self.length_freq[word_len][letter] += 1
            
            # Transition probabilities: Bigrams
            for i in range(len(word) - 1):
                self.bigram_prob[word[i]][word[i+1]] += 1
            
            # Transition probabilities: Trigrams
            for i in range(len(word) - 2):
                self.trigram_prob[word[i]][word[i+1]][word[i+2]] += 1
        
        print(f"✓ Trained on {len(words)} words")
        print(f"  Emission models: {len(self.emission_prob)} word lengths")
        print(f"  Bigrams: {sum(len(v) for v in self.bigram_prob.values())}")
        print(f"  Trigrams: {sum(sum(len(vv) for vv in v.values()) for v in self.trigram_prob.values())}")
    
    def get_emission_prob(self, masked_word, letter, available_letters):
        """Calculate emission probability P(letter | positions)"""
        word_len = len(masked_word)
        
        if word_len not in self.emission_prob:
            return 0.0
        
        prob = 0.0
        count_positions = 0
        
        # Average probability across unknown positions
        for pos, char in enumerate(masked_word):
            if char == '_' and pos in self.emission_prob[word_len]:
                pos_counter = self.emission_prob[word_len][pos]
                total = sum(pos_counter.values())
                if total > 0:
                    # Laplace smoothing
                    prob += (pos_counter[letter] + self.smoothing) / (total + self.smoothing * 26)
                    count_positions += 1
        
        return prob / count_positions if count_positions > 0 else 0.0
    
    def get_transition_prob(self, masked_word, letter, available_letters):
        """Calculate transition probability using bigrams/trigrams"""
        prob = 0.0
        contexts = 0
        
        # Check bigram context
        for i, char in enumerate(masked_word):
            if char == '_':
                # Left context
                if i > 0 and masked_word[i-1] != '_':
                    left = masked_word[i-1]
                    total = sum(self.bigram_prob[left].values())
                    if total > 0:
                        prob += (self.bigram_prob[left][letter] + self.smoothing) / (total + self.smoothing * 26)
                        contexts += 1
                
                # Right context
                if i < len(masked_word) - 1 and masked_word[i+1] != '_':
                    right = masked_word[i+1]
                    # Find letters that come before 'right'
                    for l in available_letters:
                        total = sum(self.bigram_prob[l].values())
                        if total > 0 and l == letter:
                            prob += (self.bigram_prob[l][right] + self.smoothing) / (total + self.smoothing * 26)
                            contexts += 1
                            break
        
        # Check trigram context
        for i in range(1, len(masked_word) - 1):
            if masked_word[i] == '_':
                left1 = masked_word[i-1] if i > 0 else None
                right1 = masked_word[i+1] if i < len(masked_word) - 1 else None
                
                if left1 and left1 != '_' and right1 and right1 != '_':
                    # Middle position in trigram
                    if left1 in self.trigram_prob and letter in self.trigram_prob[left1]:
                        total = sum(self.trigram_prob[left1][letter].values())
                        if total > 0:
                            prob += (self.trigram_prob[left1][letter][right1] + self.smoothing) / (total + self.smoothing * 26)
                            contexts += 1
        
        return prob / contexts if contexts > 0 else 0.0
    
    def get_letter_probabilities(self, masked_word, guessed_letters):
        """Get probability distribution for next letter"""
        available_letters = set('abcdefghijklmnopqrstuvwxyz') - guessed_letters
        word_len = len(masked_word)
        
        if not available_letters:
            return {}
        
        probabilities = {}
        
        for letter in available_letters:
            # Emission probability (from HMM states)
            emission = self.get_emission_prob(masked_word, letter, available_letters)
            
            # Transition probability (from N-grams)
            transition = self.get_transition_prob(masked_word, letter, available_letters)
            
            # Length-specific frequency
            length_prob = 0.0
            if word_len in self.length_freq:
                total = sum(self.length_freq[word_len].values())
                if total > 0:
                    length_prob = (self.length_freq[word_len][letter] + self.smoothing) / (total + self.smoothing * 26)
            
            # Global frequency
            global_prob = 0.0
            total_global = sum(self.unigram_freq.values())
            if total_global > 0:
                global_prob = (self.unigram_freq[letter] + self.smoothing) / (total_global + self.smoothing * 26)
            
            # Adaptive weighting based on revealed ratio
            revealed_ratio = sum(1 for c in masked_word if c != '_') / len(masked_word)
            
            if revealed_ratio < 0.2:  # Early game
                probabilities[letter] = 0.1 * emission + 0.1 * transition + 0.3 * length_prob + 0.5 * global_prob
            elif revealed_ratio < 0.5:  # Mid game
                probabilities[letter] = 0.25 * emission + 0.25 * transition + 0.3 * length_prob + 0.2 * global_prob
            else:  # Late game
                probabilities[letter] = 0.3 * emission + 0.4 * transition + 0.2 * length_prob + 0.1 * global_prob
        
        # Normalize
        total = sum(probabilities.values())
        if total > 0:
            probabilities = {k: v/total for k, v in probabilities.items()}
        
        return probabilities

print("✓ ImprovedHMM class defined")

## 3. Train the Improved HMM

In [ ]:
# Train improved HMM
improved_hmm = ImprovedHMM(smoothing=0.1)
improved_hmm.train(corpus)

## 4. Improved HMM Agent

In [ ]:
class ImprovedHMMAgent:
    """Hangman agent using Improved HMM"""
    
    def __init__(self, hmm_model):
        self.hmm = hmm_model
        self.vowels = set('aeiou')
        
    def get_action(self, masked_word, guessed_letters, lives_remaining):
        """Select next letter to guess"""
        available = set('abcdefghijklmnopqrstuvwxyz') - guessed_letters
        
        if not available:
            return None
        
        # Get probabilities from HMM
        probs = self.hmm.get_letter_probabilities(masked_word, guessed_letters)
        
        revealed_ratio = sum(1 for c in masked_word if c != '_') / len(masked_word)
        
        # Early game: prioritize vowels
        if revealed_ratio < 0.15:
            available_vowels = available & self.vowels
            if available_vowels:
                # Boost vowel probabilities
                for vowel in available_vowels:
                    if vowel in probs:
                        probs[vowel] *= 2.0
        
        # Critical situation: be conservative
        if lives_remaining <= 2 and probs:
            # Filter very low probabilities
            high_conf = {k: v for k, v in probs.items() if v > 0.05}
            if high_conf:
                return max(high_conf.items(), key=lambda x: x[1])[0]
        
        # Normal selection
        if probs:
            return max(probs.items(), key=lambda x: x[1])[0]
        
        # Fallback
        default_order = 'etaoinshrdlcumwfgypbvkjxqz'
        for letter in default_order:
            if letter in available:
                return letter
        
        return list(available)[0] if available else None

print("✓ ImprovedHMMAgent class defined")

## 5. Evaluation

In [ ]:
def play_game(word, agent, max_lives=6, verbose=False):
    """Play a single Hangman game"""
    masked = '_' * len(word)
    guessed = set()
    lives = max_lives
    wrong_guesses = 0
    
    if verbose:
        print(f"\nWord: {word}")
        print(f"Start: {masked}")
    
    while lives > 0 and '_' in masked:
        letter = agent.get_action(masked, guessed, lives)
        
        if letter is None:
            break
        
        guessed.add(letter)
        
        if letter in word:
            masked = ''.join(word[i] if word[i] == letter or masked[i] != '_' else '_' 
                            for i in range(len(word)))
            if verbose:
                print(f"✓ {letter}: {masked}")
        else:
            lives -= 1
            wrong_guesses += 1
            if verbose:
                print(f"✗ {letter} (Lives: {lives})")
    
    won = '_' not in masked
    if verbose:
        print(f"Result: {'WON' if won else 'LOST'} (Wrong: {wrong_guesses})")
    
    return {
        'won': won,
        'wrong_guesses': wrong_guesses,
        'total_guesses': len(guessed)
    }

def evaluate_agent(agent, test_words, max_lives=6):
    """Evaluate agent on all test words"""
    results = []
    
    for word in tqdm(test_words, desc="Evaluating"):
        result = play_game(word, agent, max_lives=max_lives, verbose=False)
        results.append(result)
    
    wins = sum(1 for r in results if r['won'])
    total_wrong = sum(r['wrong_guesses'] for r in results)
    avg_wrong = total_wrong / len(results)
    success_rate = wins / len(results)
    
    final_score = (success_rate * 2000) - (total_wrong * 5)
    
    return {
        'num_games': len(results),
        'wins': wins,
        'success_rate': success_rate,
        'total_wrong_guesses': total_wrong,
        'avg_wrong_guesses': avg_wrong,
        'final_score': final_score,
        'results': results
    }

print("✓ Evaluation functions defined")

In [ ]:
# Test on sample words
agent = ImprovedHMMAgent(improved_hmm)

print("Testing on sample words:")
for word in test_words[:5]:
    result = play_game(word, agent, verbose=True)

In [ ]:
# Full evaluation
print("\nEvaluating Improved HMM on full test set...")
results = evaluate_agent(agent, test_words)

print("\n" + "="*60)
print("IMPROVED HMM EVALUATION RESULTS")
print("="*60)
print(f"Total Games: {results['num_games']}")
print(f"Wins: {results['wins']} ({results['success_rate']:.2%})")
print(f"Total Wrong Guesses: {results['total_wrong_guesses']}")
print(f"Avg Wrong Guesses: {results['avg_wrong_guesses']:.3f}")
print(f"\nFINAL SCORE: {results['final_score']:.2f}")
print("="*60)

print("\nComparison:")
print("-" * 60)
print(f"Original HMM:   19.80% success, -55,324 score")
print(f"RL + HMM:       19.90% success, -55,302 score")
print(f"Improved HMM:   {results['success_rate']:.2%} success, {results['final_score']:.0f} score")
print("-" * 60)

## 6. Save Improved Model

In [ ]:
# Save model data (convert defaultdicts to dicts)
model_data = {
    'emission_prob': improved_hmm.emission_prob,
    'bigram_prob': {k: dict(v) for k, v in improved_hmm.bigram_prob.items()},
    'trigram_prob': {k: {k2: dict(v2) for k2, v2 in v.items()} 
                     for k, v in improved_hmm.trigram_prob.items()},
    'unigram_freq': dict(improved_hmm.unigram_freq),
    'length_freq': improved_hmm.length_freq,
    'smoothing': improved_hmm.smoothing
}

with open('improved_hmm_model.pkl', 'wb') as f:
    pickle.dump(model_data, f)

print("✓ Model saved to 'improved_hmm_model.pkl'")

## 7. Summary

### Key Improvements to HMM:

1. **N-gram Transition Probabilities**
   - Added bigram and trigram models
   - Models letter sequence patterns
   - Helps predict next letter based on context

2. **Laplace Smoothing**
   - Handles unseen patterns gracefully
   - Prevents zero probabilities
   - Improves generalization

3. **Context-Aware Predictions**
   - Uses revealed letters for better predictions
   - Bigram context (left/right neighbors)
   - Trigram context (surrounding letters)

4. **Adaptive Weighting**
   - Different strategies for early/mid/late game
   - Balances emission and transition probabilities
   - Adjusts based on revealed_ratio

5. **Strategic Letter Selection**
   - Vowel prioritization in early game
   - Conservative selection when lives are low
   - Smart fallback strategy

### Expected Improvement:
- Original HMM: ~20% success
- **Improved HMM: 28-32% success** (estimated)
- Still maintains HMM framework as required
- Adds transition probabilities (key HMM component)